In [1]:
from qiskit import QuantumCircuit, Aer, execute, transpile
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
from datetime import datetime

def identify_search_pairs(solutions):
    num_qubits = len(solutions[0])
    pairs = []
    
    for i in range(0, num_qubits, 2):
        if i + 1 < num_qubits:  # Ensure we don't go out of bounds
            differing = any(solution[i] != solutions[0][i] or solution[i+1] != solutions[0][i+1] for solution in solutions)
            if differing:
                pairs.append((i, i+1, [solution[i:i+2] for solution in solutions]))
    
    return pairs

def setup_circuit(n):
    """ Prepare a new quantum circuit with ancilla initialized. """
    circuit = QuantumCircuit(n + 1, n)  # n qubits + 1 ancilla
    circuit.x(n)  # Set ancilla to |1>
    circuit.h(n)  # Apply Hadamard to ancilla
    return circuit

def search_and_reset(n, solutions, pairs):
    results = {}
    backend = Aer.get_backend('aer_simulator')
    first_solution = True

    for i, solution in enumerate(solutions):
        circuit = setup_circuit(n)  # Reset the entire circuit for each solution
        searched_qubits = set()
        
        if not first_solution:
            # Apply Grover's search only on differing segments for subsequent solutions
            time1 = datetime.now()
            previous_solution = solutions[i - 1]
            for start, end, target_states in pairs:
                # Check if this segment needs to be re-searched
                if solution[start:end+1] != previous_solution[start:end+1]:
                    target_state = solution[start:end+1]
                    qubits_to_superpose = list(range(start, end + 1))
                    partial_hadamard_transform(circuit, qubits_to_superpose)
                    partial_oracle(circuit, qubits_to_superpose, n, target_state)
                    partial_diffuser(circuit, qubits_to_superpose)
                    searched_qubits.update(qubits_to_superpose)
            time2 = datetime.now()
            print(time2-time1)
                
            # Manually adjust the common qubits to match the previous solution
            for j in range(n):
                if j not in searched_qubits and solution[j] == previous_solution[j]:
                    if solution[j] == '1':
                        circuit.x(j)
        
        else:
            # Apply Grover's search in steps for the first complete solution
            time1 = datetime.now()
            for step in range(0, n, 2):
                qubits_to_superpose = list(range(step, min(step + 2, n)))
                partial_hadamard_transform(circuit, qubits_to_superpose)
                partial_oracle(circuit, qubits_to_superpose, n, solution[step:step + 2 if step + 2 <= n else n])
                partial_diffuser(circuit, qubits_to_superpose)
                searched_qubits.update(qubits_to_superpose)
            time2 = datetime.now()
            print(time2-time1)
            first_solution = False  
        
        # Measure only the target qubits, not the ancilla
        circuit.measure(list(range(n)), list(range(n)))
        transpiled_circuit = transpile(circuit, backend)
        result = execute(transpiled_circuit, backend, shots=1024).result()
        counts = result.get_counts()
        results[solution] = counts
        
        # Remove the measurement instructions to continue using the same circuit for the next iteration
        circuit.data = circuit.data[:-n]
        
        #print(circuit)
    
    return results

def partial_hadamard_transform(circuit, qubits):
    for qubit in qubits:
        circuit.h(qubit)

def partial_oracle(circuit, qubits, ancilla, target_state):
    for qubit, bit in zip(qubits, target_state):
        if bit == '0':
            circuit.x(qubit)
    circuit.mcx(qubits, ancilla)
    for qubit, bit in zip(qubits, target_state):
        if bit == '0':
            circuit.x(qubit)

def partial_diffuser(circuit, qubits):
    for qubit in qubits:
        circuit.h(qubit)
        circuit.x(qubit)
    circuit.h(qubits[-1])
    circuit.mcx(qubits[:-1], qubits[-1])
    circuit.h(qubits[-1])
    for qubit in qubits:
        circuit.x(qubit)
        circuit.h(qubit)

# Parameters and execution
n = 8
solutions = ['00000000', '11111111']

pairs = identify_search_pairs(solutions)
result_counts = search_and_reset(n, solutions, pairs)
print(pairs)

# Display results
for solution, counts in result_counts.items():
    print(f"Results for {solution}: {counts}")
    plot_histogram(counts)
    plt.show()


0:00:00.001879
0:00:00.001495
[(0, 1, ['00', '11']), (2, 3, ['00', '11']), (4, 5, ['00', '11']), (6, 7, ['00', '11'])]
Results for 00000000: {'00000000': 1024}
Results for 11111111: {'11111111': 1024}
